In [55]:
# 패키지 설치
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split

# settings : 결과로 바로 그림이 나오도록 + 스타일 서식
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 
sns.set(style="darkgrid") 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [56]:
# raw data 로딩 
raw_data = pd.read_csv(r'C:\Users\user\Desktop\intern.csv')
raw_data.head()

,Unnamed: 0,ORD_NO,H_YMD,O_YMD,PKG_GOODS_NO,PKG_GOODS_NM,GOODS_NO,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,RECVR_ROAD_BASE_ADDR,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
0,0,202201095519105,2022-01-11 00:00:00,2022-01-09 12:26:10,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,충청북도 증평군 증평읍 송산로 11 (지평더웰아파트),6900,569.00,6331
1,1,202201105522898,2022-01-11 00:00:00,2022-01-10 10:35:49,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,서울특별시 강남구 논현로85길 52 (역삼동),6900,177.00,6723
2,2,202201115527172,2022-01-12 00:00:00,2022-01-11 11:38:32,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"서울특별시 송파구 올림픽로4길 42 (잠실동, 우성아파트)",6900,221.00,6679
3,3,202201105523467,2022-01-12 00:00:00,2022-01-10 13:21:51,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,강원도 속초시 도리원길 11-14 (노학동),6900,128.00,6772
4,4,202201105522519,2022-01-11 00:00:00,2022-01-10 08:16:21,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"경기도 용인시 수지구 동천로 64 (동천동, 동천마을동문굿모닝힐5차아파트)",6900,157.00,6743


In [215]:
weather = pd.read_excel(r'C:\Users\user\Desktop\w.xlsx')
weather.rename(columns={'date':'H_YMD'}, inplace=True)
weather['H_YMD'] = pd.to_datetime(weather['H_YMD'])
weather = weather.sort_values(by = 'H_YMD')
weather.drop('spot', axis=1, inplace=True)

weather

,Unnamed: 0,H_YMD,temp,rain,humid
10824,10824,2019-12-31,-3.60,0.00,51.80
10104,10104,2019-12-31,-7.90,0.00,39.30
10224,10224,2019-12-31,-4.70,0.00,55.00
9864,9864,2019-12-31,-2.50,0.00,33.00
9744,9744,2019-12-31,-2.20,0.00,46.10
...,...,...,...,...,...
15560,15560,2022-07-11,3.83,0.00,54.58
15554,15554,2022-07-11,2.79,0.00,44.79
15578,15578,2022-07-11,2.54,0.00,39.38
15575,15575,2022-07-11,2.42,0.00,54.79


In [216]:
# 카피 데이터 생성
copy_data = raw_data.copy()

# 희망배송일 순서대로 데이터를 정렬
copy_data = copy_data.sort_values(by = 'H_YMD')


copy_data.head()

,Unnamed: 0,ORD_NO,H_YMD,O_YMD,PKG_GOODS_NO,PKG_GOODS_NM,GOODS_NO,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,RECVR_ROAD_BASE_ADDR,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
166288,119348,201912302355791,2020-01-01 00:00:00,2019-12-30 21:59:40,단품,단품,14375,옛날잡채(500g),메인요리,1,0,0,1,"경기도 수원시 장안구 경수대로976번길 22 (조원동, 수원 한일타운)",8600,1408.00,7192
37753,37753,201912302355557,2020-01-01 00:00:00,2019-12-30 20:50:07,단품,단품,1901012353,수제계란말이(350g),반찬,2,0,0,2,서울 성북구 동소문로34길 24 삼성아파트,6300,NaN,6300
162585,115645,201912302353567,2020-01-01 00:00:00,2019-12-30 11:02:39,단품,단품,1823,숙주나물(300g),반찬,1,0,0,1,"서울특별시 마포구 삼개로 33 (도화동, 도화3지구우성아파트)",3300,486.00,2814
164041,117101,201912302354611,2020-01-01 00:00:00,2019-12-30 15:19:58,단품,단품,1712010310,두메산나물비빔밥재료,메인요리,1,0,0,1,경기도 용인시 수지구 문인로3번길 22 (풍덕천동),6900,264.00,6636
164040,117100,201912302352965,2020-01-01 00:00:00,2019-12-30 10:08:58,단품,단품,1712010310,두메산나물비빔밥재료,메인요리,1,0,0,1,"경기도 구리시 아차산로487번길 27 (교문동, 아차산어울림아파트)",6900,317.00,6583


In [217]:
# 희망배송일 변수를 datetime 형식으로 변환

copy_data['H_YMD'] = pd.to_datetime(copy_data['H_YMD'])

## 전처리

In [218]:
# 예측 대상에는 없는 상품들은 모두 nan값으로 변경
copy_data['GOODS_NM'].loc[copy_data['GOODS_NM'] == '꼬막무침 (250g)'] = '꼬막무침 (260g)'
copy_data['GOODS_NM'].loc[copy_data['GOODS_NM'] == '고소한도토리묵무침(265g)'] = '고소한도토리묵무침(360g)'
copy_data['GOODS_NM'].loc[copy_data['GOODS_NM'] == '건고사리나물볶음(150g)'] = '고사리나물볶음(150g)'

In [219]:
# 내일한정, 한정판매는 단품으로 대체
# 한정판매 중에는 단품이 아닌 세트도 있어서 이를 유의해야 함(ex. '[한정판매]손님초대상 set', '[한정판매]반찬 한 상 set', '[한정판매]키즈 set')

# 내일한정이 붙은 제품은 모두 단품인 것을 확인하여 카테고리를 단품으로 바꿔줌, 
# [내일한정] 고소한도토리묵무침(265g), [내일한정] 건고사리나물볶음(150g), [내일한정] 꼬막무침 (250g)은 제외
copy_data['PKG_GOODS_NM'].loc[copy_data['PKG_GOODS_NM'].str.contains('내일한정')] = '단품'

# 한정판매 중에서 두 제품만 단품이었기에 두 제품을 단품으로 바꿔줌
copy_data['PKG_GOODS_NM'].loc[copy_data['PKG_GOODS_NM'] == '[한정판매] 두메산나물비빔밥재료'] = '단품'  
copy_data['PKG_GOODS_NM'].loc[copy_data['PKG_GOODS_NM'] == '[한정판매] 옛날잡채(500g)'] = '단품'

# 단품으로 바꿔주지 않은 모든 제품을 세트로 바꿔줌
copy_data['PKG_GOODS_NM'].loc[copy_data['PKG_GOODS_NM'] != '단품'] = '세트'

In [220]:
# 할인액의 NaN값을 모두 0으로 대체
# 할인되지 않은 상품을 산것으로 추측되어 이 경우 할인액은 0이 되는게 맞다.

copy_data['DISCOUNT_AMT'].fillna(0, inplace=True)

In [209]:
# 불필요한 컬럼들 제거
# Unnamed: 단순 데이터 순서를 나타내는 컬럼이기에 삭제
# O_YMD: 주문 시간은 오더 건수와 큰 상관이 없기에 삭제
# ORD_NO: 주문 내용의 일련번호이기에 삭제
# PKG_GOODS_NO: PKG_GOODS_NM과 내용이 겹치고 잘못 적재된 데이터이기에 삭제
# RECVR_ROAD_BASE_ADDR: 비대면으로 주문하는 시대에 지역정보는 큰 의미가 없을 것 같아 삭제

dummy_col = ['Unnamed: 0', 'GOODS_NO', 'O_YMD', 'ORD_NO', 'PKG_GOODS_NO', 'RECVR_ROAD_BASE_ADDR']
copy_data = copy_data.drop(dummy_col, axis=1)

copy_data.head()

,H_YMD,PKG_GOODS_NM,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
166288,2020-01-01,단품,옛날잡채(500g),메인요리,1,0,0,1,8600,1408.00,7192
37753,2020-01-01,단품,수제계란말이(350g),반찬,2,0,0,2,6300,0.00,6300
162585,2020-01-01,단품,숙주나물(300g),반찬,1,0,0,1,3300,486.00,2814
164041,2020-01-01,단품,두메산나물비빔밥재료,메인요리,1,0,0,1,6900,264.00,6636
164040,2020-01-01,단품,두메산나물비빔밥재료,메인요리,1,0,0,1,6900,317.00,6583


## 피쳐 엔지니어링

In [226]:
# 할인율을 나타내는 SALE_PERCENTAGE 변수 생성
# 할인액 / 판매가격 * 100

merge['SALE_PERCETANGE'] = (merge['DISCOUNT_AMT'] / merge['SALE_PRICE']) * 100

In [227]:
# 희망배송일의 연도,월,일,시간,분,초를 각각 컬럼으로 만들어줌

merge['year'] = merge['H_YMD'].dt.year
merge['month'] = merge['H_YMD'].dt.month
merge['day'] = merge['H_YMD'].dt.day
# copy_data['hour'] = copy_data['H_YMD'].dt.hour
# copy_data['minute'] = copy_data['H_YMD'].dt.minute
# copy_data['second'] = copy_data['H_YMD'].dt.second


In [228]:
def outlier(df,col):
    q1 = df[col].quantile(0.25) # 1사분위
    q3 = df[col].quantile(0.75) # 3사분위 
    iqr = q3 - q1               # iqr 수치

    df.loc[(df[col] > (q3 + 1.5 * iqr)) | (df[col] < (q1 - 1.5 * iqr)), col] = df[col].median() # 이상치를 중앙값으로 대체

ol_col = ['ORD_QTY', 'CANCEL_QTY', 'RET_QTY', 'REAL_ORD_QTY', 'SALE_PRICE', 'FINAL_PRICE']

for i in ol_col:
    #print(i)
    outlier(merge, i)

In [233]:
cols = ['PKG_GOODS_NM', 'GOODS_NM', 'STD_GSGR_NO_LEV1_NM']

merge= pd.get_dummies(merge, columns = cols, drop_first=False)

In [235]:
# 데이터를 주 단위로 groupby 하고 해당 주에 팔린 상품의 개수를 sum()한다.

resampled = merge.resample(rule='1D', on='H_YMD')
resampled.sum()

merge = resampled.sum()

## 데이터 분할

In [241]:
# 데이터의 컬럼에 '['나 ','같은 특수기호가 들어가 있으면 학습이 안되기 때문에 제거해준다.
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

merge.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in merge.columns.values]

In [150]:
H_YMD = copy_data['H_YMD']

copy_data.drop('H_YMD', axis=1, inplace=True)

KeyError: 'H_YMD'

In [244]:
# 만들어진 데이터 기준 121행부터 2022년 5월의 데이터이므로 2022년 5월 이후 데이터를 test 데이터로 사용한다
# 104: 2022 전체 데이터
# 121: 2022.05 데이터

# train_data = copy_data[:104]
# test_data = copy_data[104:]

# train_data = copy_data[:121]
# test_data = copy_data[121:]

train_data = merge[:-31]
test_data = merge[-31:]

# train_data = copy_data[:-26567]
# test_data = copy_data[-26567:]

test_data.head()

,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE,temp,rain,humid,...,GOODS_NM_순살코다리강정(180g),GOODS_NM_양장피,GOODS_NM_열무비빔밥재료믹스(2인분),GOODS_NM_옛날잡채(500g),GOODS_NM_우삼겹숙주볶음(250g),GOODS_NM_채소계란찜(340g),GOODS_NM_한돈 제육볶음(700g),STD_GSGR_NO_LEV1_NM_국,STD_GSGR_NO_LEV1_NM_메인요리,STD_GSGR_NO_LEV1_NM_반찬
H_YMD,,,,,,,,,,,,,,,,,,,,,
2022-05-01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2022-05-02,19346.00,0.00,0.00,19346.00,136312800.00,13941020.00,122371780.00,260317.50,14338.80,1069321.70,...,884.00,1190.00,0.00,2142.00,255.00,1581.00,850.00,561.00,5746.00,13039.00
2022-05-03,15028.00,0.00,0.00,15028.00,107623600.00,7695917.00,99927683.00,214502.60,0.00,786936.80,...,374.00,765.00,0.00,1700.00,374.00,1292.00,884.00,612.00,4760.00,9656.00
2022-05-04,18496.00,0.00,0.00,18496.00,131466100.00,8324356.00,123141744.00,320524.80,0.00,881062.40,...,442.00,1088.00,0.00,2550.00,408.00,1275.00,867.00,578.00,6375.00,11543.00
2022-05-05,8993.00,0.00,0.00,8993.00,63830750.00,5776260.00,58054490.00,168354.25,0.00,488716.65,...,306.00,340.00,0.00,1173.00,187.00,476.00,255.00,255.00,2720.00,6018.00


In [245]:
# 타겟이 되는 20개 제품을 설정한다

target = ['GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g)',
       'GOODS_NM_가정집 오징어불고기/셀프(380g)', 'GOODS_NM_고사리나물볶음(150g)',
       'GOODS_NM_건표고버섯볶음', 'GOODS_NM_고구마 품은 라자냐(450g)',
       'GOODS_NM_고소한도토리묵무침(360g)', 'GOODS_NM_꼬막무침 (260g)',
       'GOODS_NM_두메산나물비빔밥재료', 'GOODS_NM_메밀소바(2인분)', 'GOODS_NM_셀프두부조림(600g)',
       'GOODS_NM_소고기유니짜장소스(1인분, 200g)', 'GOODS_NM_수제계란말이(350g)',
       'GOODS_NM_숙주나물(300g)', 'GOODS_NM_순살코다리강정(180g)', 'GOODS_NM_양장피',
       'GOODS_NM_열무비빔밥재료믹스(2인분)', 'GOODS_NM_옛날잡채(500g)',
       'GOODS_NM_우삼겹숙주볶음(250g)', 'GOODS_NM_채소계란찜(340g)',
       'GOODS_NM_한돈 제육볶음(700g)']

print(len(target)) # // 20

# 학습&검증 데이터와 테스트 데이터를 나눈다.
# X, y -> 학습 및 검증데이터
# X_test, y_test -> 테스트데이터 
X, X_test = train_data.drop(target, axis=1), test_data.drop(target, axis=1)
y, y_test = train_data[target], test_data[target]

20


In [246]:
# train_test_split 메서드를 사용하여 학습 검증 데이터를 7:3비율로 나눈다

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = .3, shuffle=False, random_state = 2022)

In [247]:
# 학습데이터 모양 확인
X_train.shape

(595, 19)

In [248]:
X_train.head()

,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE,temp,rain,humid,SALE_PERCETANGE,year,month,day,PKG_GOODS_NM_단품,PKG_GOODS_NM_세트,STD_GSGR_NO_LEV1_NM_국,STD_GSGR_NO_LEV1_NM_메인요리,STD_GSGR_NO_LEV1_NM_반찬
H_YMD,,,,,,,,,,,,,,,,,,,
2020-01-01,5525.00,0.00,0.00,5525.00,38567900.00,1538092.00,37029808.00,-471.25,32.50,332231.25,21837.28,11160500,5525,5525,5525.00,0.00,0.00,3196.00,2329.00
2020-01-02,8619.00,0.00,0.00,8619.00,58837000.00,2331465.00,56505535.00,22612.20,0.00,573264.90,34257.07,17410380,8619,17238,8619.00,0.00,0.00,4777.00,3842.00
2020-01-03,11101.00,0.00,0.00,11101.00,77968800.00,1553936.00,76414864.00,35588.50,0.00,689502.70,22724.39,22424020,11101,33303,11101.00,0.00,0.00,5882.00,5219.00
2020-01-04,12274.00,0.00,0.00,12274.00,84323400.00,2150755.00,82172645.00,35414.10,0.00,740699.80,30982.86,24793480,12274,49096,12206.00,68.00,0.00,5695.00,6579.00
2020-01-05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,0.00,0.00


### 모델링

In [249]:
# 범위가 제각각인 각 컬럼들을 0~1 사이에 범위로 조정해준다.

from sklearn.preprocessing import MinMaxScaler
col_x = list(X)
scaler = MinMaxScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)

In [250]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance

%matplotlib inline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [251]:
xgb_model = XGBRegressor(
    booster='gblinear',
    n_estimators=1000, 
    learning_rate=0.1, 
    gamma=1, 
    subsample=0.75,
    colsample_bytree=1, 
    max_depth=5
    )

In [252]:
xgb_model.fit(
    X_train_s,
    y_train, 
    eval_metric="rmse",
    eval_set=[(X_train_s, y_train), (X_val_s, y_val)], 
    early_stopping_rounds=30
    )

[15:29:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytree", "gamma", "max_depth", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:576.82190	validation_1-rmse:543.32321
[1]	validation_0-rmse:514.90157	validation_1-rmse:510.28169
[2]	validation_0-rmse:503.02345	validation_1-rmse:512.80945
[3]	validation_0-rmse:495.11416	validation_1-rmse:513.27324
[4]	validation_0-rmse:488.61765	validation_1-rmse:512.65480
[5]	validation_0-rmse:483.15191	validation_1-rmse:512.26583
[6]	validation_0-rmse:478.53518	validation_1-rmse:511.76503
[7]	validation_0-rmse:474.60159	validation_1-rmse:511.23358
[8]	validation_0-rmse:471.20157	validation_1-rmse:510.83072


XGBRegressor(base_score=0.5, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=1, gpu_id=-1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=0, num_parallel_tree=None,
             predictor=None, random_state=0, reg_alpha=0, reg_lambda=0, ...)

In [253]:
xgb_pred = xgb_model.predict(X_val_s)

xgb_pred

array([[ 582.0663 ,  990.069  ,  873.05383, ...,  473.31342, 1231.6263 ,
         840.3208 ],
       [ 473.51392,  794.99756,  694.2898 , ...,  444.22614, 1028.1907 ,
         729.7008 ],
       [ 690.3642 ,  757.0711 ,  679.31824, ...,  458.28998,  961.1804 ,
        1048.3885 ],
       ...,
       [ 394.41162,  712.8685 ,  603.3444 , ...,  569.8515 , 1016.1044 ,
         872.60486],
       [ 778.4419 ,  712.50415,  644.68494, ...,  801.1428 , 1120.138  ,
        1320.6768 ],
       [ 624.4029 ,  762.41785,  663.46655, ...,  758.1009 , 1096.352  ,
        1230.1506 ]], dtype=float32)

In [254]:
# 모델의 성능을 점검하는 지표로 r2와 RMSE를 사용한다.
# r2는 회귀모델에서 독립변수가 종속변수를 얼마만큼 설명해주는지 가리키는 지표이기에 사용했다.
# 회귀 예측에 대한 정확도는 accuracy로 판단할 수 없기 때문에 rmse를 사용했다.

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

r2 = r2_score(y_val, xgb_pred)
RMSE = mean_squared_error(y_val, xgb_pred, squared=False)

print('r2: ', r2)
print('RMSE: ', RMSE)

r2:  -0.29350760753961963
RMSE:  416.98398758478044


In [255]:
# 테스트 데이터에 대해서 결과를 출력한다.

xgb_test_pred = xgb_model.predict(X_test_s)
df = y_test.copy()
#xgb_test_pred = pd.DataFrame(xgb_test_pred)
df[['GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g)', 'GOODS_NM_가정집 오징어불고기/셀프(380g)',
       'GOODS_NM_고사리나물볶음(150g)', 'GOODS_NM_건표고버섯볶음',
       'GOODS_NM_고구마 품은 라자냐(450g)', 'GOODS_NM_고소한도토리묵무침(360g)',
       'GOODS_NM_꼬막무침 (260g)', 'GOODS_NM_두메산나물비빔밥재료', 'GOODS_NM_메밀소바(2인분)',
       'GOODS_NM_셀프두부조림(600g)', 'GOODS_NM_소고기유니짜장소스(1인분, 200g)',
       'GOODS_NM_수제계란말이(350g)', 'GOODS_NM_숙주나물(300g)',
       'GOODS_NM_순살코다리강정(180g)', 'GOODS_NM_양장피', 'GOODS_NM_열무비빔밥재료믹스(2인분)',
       'GOODS_NM_옛날잡채(500g)', 'GOODS_NM_우삼겹숙주볶음(250g)', 'GOODS_NM_채소계란찜(340g)',
       'GOODS_NM_한돈 제육볶음(700g)']] = xgb_test_pred

xgb_test_pred = df.copy()
xgb_test_pred.head()

,GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g),GOODS_NM_가정집 오징어불고기/셀프(380g),GOODS_NM_고사리나물볶음(150g),GOODS_NM_건표고버섯볶음,GOODS_NM_고구마 품은 라자냐(450g),GOODS_NM_고소한도토리묵무침(360g),GOODS_NM_꼬막무침 (260g),GOODS_NM_두메산나물비빔밥재료,GOODS_NM_메밀소바(2인분),GOODS_NM_셀프두부조림(600g),"GOODS_NM_소고기유니짜장소스(1인분, 200g)",GOODS_NM_수제계란말이(350g),GOODS_NM_숙주나물(300g),GOODS_NM_순살코다리강정(180g),GOODS_NM_양장피,GOODS_NM_열무비빔밥재료믹스(2인분),GOODS_NM_옛날잡채(500g),GOODS_NM_우삼겹숙주볶음(250g),GOODS_NM_채소계란찜(340g),GOODS_NM_한돈 제육볶음(700g)
H_YMD,,,,,,,,,,,,,,,,,,,,
2022-05-01,-27.05,78.85,57.88,-4.68,-144.30,-107.71,39.34,127.96,103.29,131.42,0.03,82.89,87.57,-56.13,-55.00,44.03,-71.15,-109.21,134.72,-106.50
2022-05-02,440.62,707.43,652.47,752.11,604.41,654.85,917.02,4527.52,411.76,1540.59,-0.02,1131.84,1165.82,401.73,496.17,353.50,1414.45,678.61,1170.38,959.87
2022-05-03,215.11,624.23,547.79,579.36,360.98,444.61,826.46,3477.71,323.14,1215.42,-0.02,853.48,943.51,245.01,376.83,267.86,1594.81,400.66,919.30,504.74
2022-05-04,230.83,759.30,658.44,684.25,403.12,545.94,967.62,4271.71,484.17,1420.25,-0.04,1002.54,1108.49,274.03,449.02,391.77,2121.65,435.04,1078.48,571.86
2022-05-05,234.90,398.33,345.25,363.48,180.42,241.60,490.49,2068.22,241.57,800.66,-0.01,584.36,604.86,140.20,206.68,175.54,657.76,238.42,630.49,376.41


In [256]:
y_test.head()

,GOODS_NM__심방골주부X더반찬_ 시골 돼지짜글이(600g),GOODS_NM_가정집 오징어불고기/셀프(380g),GOODS_NM_고사리나물볶음(150g),GOODS_NM_건표고버섯볶음,GOODS_NM_고구마 품은 라자냐(450g),GOODS_NM_고소한도토리묵무침(360g),GOODS_NM_꼬막무침 (260g),GOODS_NM_두메산나물비빔밥재료,GOODS_NM_메밀소바(2인분),GOODS_NM_셀프두부조림(600g),"GOODS_NM_소고기유니짜장소스(1인분, 200g)",GOODS_NM_수제계란말이(350g),GOODS_NM_숙주나물(300g),GOODS_NM_순살코다리강정(180g),GOODS_NM_양장피,GOODS_NM_열무비빔밥재료믹스(2인분),GOODS_NM_옛날잡채(500g),GOODS_NM_우삼겹숙주볶음(250g),GOODS_NM_채소계란찜(340g),GOODS_NM_한돈 제육볶음(700g)
H_YMD,,,,,,,,,,,,,,,,,,,,
2022-05-01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2022-05-02,561.00,578.00,391.00,935.00,170.00,816.00,816.00,3536.00,0.00,1496.00,1275.00,799.00,1071.00,884.00,1190.00,0.00,2142.00,255.00,1581.00,850.00
2022-05-03,612.00,527.00,272.00,680.00,187.00,697.00,595.00,2618.00,0.00,1207.00,680.00,527.00,1037.00,374.00,765.00,0.00,1700.00,374.00,1292.00,884.00
2022-05-04,578.00,527.00,374.00,833.00,238.00,1105.00,782.00,3111.00,0.00,1173.00,1292.00,663.00,1190.00,442.00,1088.00,0.00,2550.00,408.00,1275.00,867.00
2022-05-05,255.00,340.00,357.00,476.00,153.00,459.00,306.00,2091.00,0.00,663.00,357.00,374.00,425.00,306.00,340.00,0.00,1173.00,187.00,476.00,255.00


In [118]:
# 모델의 성능을 점검하는 지표로 r2와 RMSE를 사용한다.
# r2는 회귀모델에서 독립변수가 종속변수를 얼마만큼 설명해주는지 가리키는 지표이기에 사용했다.
# 회귀 예측에 대한 정확도는 accuracy로 판단할 수 없기 때문에 rmse를 사용했다.

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

r2 = r2_score(y_test, xgb_test_pred)
RMSE = mean_squared_error(y_test, xgb_test_pred, squared=False)

print('r2: ', r2)
print('RMSE: ', RMSE)

r2:  -0.13540780907270494
RMSE:  0.22055428
